In [2]:
import chromadb
import pandas as pd
from sentence_transformers import SentenceTransformer

client = chromadb.Client()
client = chromadb.PersistentClient(path="./vector_db")
collection = client.get_or_create_collection("synthetic_clothes_data")


/Users/kyle-anthonyhay/Documents/CODE/Recature-Homework/Revature-Cognizant-Training-Exercises/December/venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
# Load CSV
df = pd.read_csv("synthetic-clothes.csv")


# Prepare data for insertion
documents = []
metadatas = []
ids = []

for idx, row in df.iterrows():
    # Create a searchable text document from product info
    # Combine product name, brand, category, and review text for semantic search
    product_text = f"{row['product_name']} {row['brand']} {row['category']}"
    if pd.notna(row['review_text']):
        product_text += f" {row['review_text']}"
    
    documents.append(product_text)
    
    # Store all other fields as metadata
    metadata = {
        'product_id': str(row['product_id']),
        'product_name': str(row['product_name']),
        'brand': str(row['brand']),
        'category': str(row['category']),
        'gender': str(row['gender']),
        'size': str(row['size']),
        'color': str(row['color']),
        'material': str(row['material']),
        'fit': str(row['fit']),
        'season': str(row['season']),
        'price_usd': float(row['price_usd']) if pd.notna(row['price_usd']) else None,
        'avg_rating': float(row['avg_rating']) if pd.notna(row['avg_rating']) else None,
    }
    
    # Add review info if available
    if pd.notna(row['review_text']):
        metadata['review_text'] = str(row['review_text'])
        metadata['review_rating'] = float(row['review_rating']) if pd.notna(row['review_rating']) else None
    
    metadatas.append(metadata)
    ids.append(f"product_{idx}")


In [4]:
# Add to collection (batch insert is more efficient)
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)
print(f"Inserted {len(ids)} products into the collection")

Inserted 100 products into the collection


In [4]:
%pip install streamlit

  Using cached streamlit-1.52.2-py3-none-any.whl.metadata (9.8 kB)
  Using cached altair-6.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached gitpython-3.1.46-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
Using cached streamlit-1.52.2-py3-none-any.whl (9.0 MB)
Using cached altair-6.0.0-py3-none-any.whl (795 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached gitpython-3.1.46-py3-none-any.whl (208 kB)
Using cached gitdb-4.0.12-py3-none-any.whl (62 kB)
Using cached pydeck-0.9.1-py2.py3-none-any.whl (6.9 MB)
Using cached smmap-5.0.2-py3-none-any.whl (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [streamlit]/8 [streamlit]
Note: you may need to restart the kernel to use updated packages.


In [5]:
import streamlit as st
import chromadb

@st.cache_resource
def get_collection():
    client = chromadb.PersistentClient(path="./vector_db")
    return client.get_or_create_collection("synthetic_clothes_data")

collection = get_collection()

st.title("Product Search")
query = st.text_input("Search products")

if query:
    results = collection.query(query_texts=[query], n_results=5)
    
    for doc, meta, dist in zip(
        results['documents'][0],
        results['metadatas'][0],
        results['distances'][0]
    ):
        st.write(f"**{meta.get('product_name')}** - {meta.get('brand')}")
        st.write(f"${meta.get('price_usd', 0):.2f} | ⭐ {meta.get('avg_rating', 0)}/5")
        st.write(f"Match: {(1-dist)*100:.1f}%")
        st.write(doc)
        st.divider()

2026-01-06 11:12:38.100 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-06 11:12:38.136 
  command:

    streamlit run /Users/kyle-anthonyhay/Documents/CODE/Recature-Homework/Revature-Cognizant-Training-Exercises/December/venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2026-01-06 11:12:38.137 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-06 11:12:38.137 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-06 11:12:38.137 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-06 11:12:38.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-06 11:12:38.243 Thread 'MainThread': missing ScriptRunContext! This war